In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re
from tqdm import tqdm

In [50]:
def get_article(url):
    html = BeautifulSoup(
        requests.get(url).content, 'html.parser'
    )
    title = html.find('h1', {'class':'detail__title'}).get_text().strip()
    date = datetime.strptime(
        html.find('div', {'class':'detail__time'}).find('time').get_text().strip(), 
        '%H:%M %d/%m/%Y'
    ).timestamp()
    desc = html.find('h2', {'class':'detail__summary'}).get_text().strip()
    categ = html.find('li', {'class':'breadcrumb-item active'}).find('a').get_text().strip()
    body = html.find('div',{'class':'detail__content'}).get_text().strip()
    return {
        'title':title, 'source':"TapChiTaiChinh", 'date':date, 'desc':desc, 'categ':categ, 'body':body
    }


In [58]:
headers = {
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-language': 'en-US,en;q=0.9,vi;q=0.8',
    # 'cookie': '_ga=GA1.1.1318016657.1730997942; __zi=3000.SSZzejyD3DyxYU6gsmyDrIU1vg6T40sV9TNuxvC8JiCtt-VzqHfOrtI0eVZGKXVIV86azfHN5jiqYQN-c5yNr3On.1; imf.log.cname=52b76bc0-999f-444e-6a4f-91358ae01376; ul_19000=true; ul_1902178=true; _ga_NPYJXLCD14=GS1.1.1731032892.2.1.1731033110.0.0.0',
    'priority': 'u=1, i',
    'referer': 'https://tapchitaichinh.vn/Chuyen-dong-tai-chinh.html',
    'sec-ch-ua': '"Chromium";v="130", "Microsoft Edge";v="130", "Not?A_Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 Edg/130.0.0.0',
    'x-requested-with': 'XMLHttpRequest',
}

catIds = [178, 76, 1107180129, 171, 65, 84, 1107182143, 87, 136, 90]
maxPage = 5

all_url = []
for catId in catIds:
    for page in range(1, maxPage + 1):
        response = requests.get(f'https://tapchitaichinh.vn/api/category/{catId}/paging/{page}', headers=headers).json()
        for link in response:
            url = ('https://tapchitaichinh.vn/' + link["Url"])
            all_url.append(url)

In [62]:
dates = []
sources = []
urls = []
titles = []
descs = []
categs = []
bodies = []

for link in tqdm(all_url):
    try:
        data = get_article(link)
        if data["date"] < datetime(2024, 10, 30).timestamp():
            next
        else:
            dates.append(data["date"])
            sources.append(data["source"])
            urls.append(link)
            titles.append(data['title'])
            descs.append(data["desc"])
            categs.append(data["categ"])
            bodies.append(data['body'])
    except Exception as error:
        print("Error at: ", link, error)

tctc_data = pd.DataFrame(
    {
        'date': dates,
        'source' : sources,
        'url' : urls,
        'title' : titles,
        'description' : descs,
        'category' : categs,
        'body' : bodies
    }
)

  5%|▌         | 20/400 [00:01<00:36, 10.55it/s]

Error at:  https://tapchitaichinh.vn//du-toan-thu-chi-ngan-sach-nha-nuoc-nam-2025.html 'NoneType' object has no attribute 'find'


 19%|█▉        | 75/400 [00:07<00:31, 10.28it/s]

Error at:  https://tapchitaichinh.vn//sua-doi-bo-sung-luat-chung-khoan-vi-loi-ich-chung-va-su-phat-trien-lanh-manh-ben-vung-cua-thi-truong.html 'NoneType' object has no attribute 'get_text'


 54%|█████▎    | 214/400 [00:21<00:18, 10.27it/s]

...

 86%|████████▌ | 342/400 [00:33<00:05, 10.89it/s]

Error at:  https://tapchitaichinh.vn//quy-dinh-ve-thanh-toan-chi-phi-thuoc-thiet-bi-y-te-cho-nguoi-co-the-bao-hiem-y-te.html 'NoneType' object has no attribute 'find'


 90%|█████████ | 361/400 [00:35<00:03, 10.56it/s]

Error at:  https://tapchitaichinh.vn//chuyen-doi-giao-thong-xanh-khoi-tao-ky-nguyen-xanh.html 'NoneType' object has no attribute 'get_text'


100%|██████████| 400/400 [00:39<00:00, 10.22it/s]


In [64]:
tctc_data.to_csv("TapChiTaiChinh_data.csv")